In [9]:
import numpy as np
from numpy import pi
from numpy import tan
from prettytable import PrettyTable


# REQUIREMENTS
REQUIRED_SPEED = 1  # m/s # The top speed of the motor.
TIME = 0.001  # s (1ms) # The time in which the motor must come to a stop from top speed.
REQUIRED_RESOLUTION = 0.1 # mm # The resolution of the motor when translated to linear motion.

# CONSTANTS
ROTATION_RANGES = np.array([30, 40, 50, 60, 70]) # Degrees of rotation allowed by the system.
RESOLUTION_IN_DEG = 0.2930 # Degrees per pulse of the servo motor.
RESOLUTION_IN_DEG = 0.9
MAX_RPM = 59 # The maximum RPM of the motor.

# TEST WIDTH ----------------
MIN_HALF_WIDTH = 0.25 / 2
# ------------------------------

# Print out constant variables
print("\n----------------- Constant variables -----------------")
constant_variables = [var_name for var_name, var_value in globals().items()
                    if var_name.isupper() and not callable(var_value)]
for var_name in constant_variables:
    print(f"{var_name}: {globals()[var_name]}")


reduction_gears = np.arange(1, 2)
reduction_gears = [1, 2, 4, 8, 16]

for rotation_range in ROTATION_RANGES:
    # rotation_range is the angle of rotation that results in 1m of linear motion
    total_rotations = np.array([rotation_range * x for x in reduction_gears])
    max_speeds = (MAX_RPM / (total_rotations / 360)) / 60 # m/s

    degrees_per_second = rotation_range * REQUIRED_SPEED  
    required_rpm = degrees_per_second / 360 * 60
    required_rpms = [required_rpm * x for x in reduction_gears]

    # Minimum distance from tank
    min_dist = MIN_HALF_WIDTH / tan((rotation_range * pi / 180) / 2) * 1000 # mm

    # Lateral step distance
    lateral_step_sizes = min_dist * tan(RESOLUTION_IN_DEG * pi / 180) # mm
    lateral_step_sizes = [lateral_step_sizes / x for x in reduction_gears]


    table = PrettyTable()
    table.field_names = ['Reduction', 'Lateral step (mm)', f'Req RPM for {REQUIRED_SPEED} m/s', 'Max speed (m/s)', 'Total rotation (deg)']
    table.align = 'r'
    table.title = f"Rotation range: {rotation_range} deg; Min width: {MIN_HALF_WIDTH} m; Resolution: {RESOLUTION_IN_DEG} deg"
    for gear, step, rpm, max_speed, rotation in zip(reduction_gears, lateral_step_sizes, required_rpms, max_speeds, total_rotations):
        # if abs(rotation - 300) <= 50:
        # if rotation <= 300 and step <= 1.5:
        table.add_row([gear, round(step, 2), round(rpm, 2), round(max_speed, 3), rotation])

    # Display the table
    print(table)


----------------- Constant variables -----------------
REQUIRED_SPEED: 1
TIME: 0.001
REQUIRED_RESOLUTION: 0.1
ROTATION_RANGES: [30 40 50 60 70]
RESOLUTION_IN_DEG: 0.9
MAX_RPM: 59
MIN_HALF_WIDTH: 0.125
+--------------------------------------------------------------------------------------------+
|                                   Rotation range: 30 deg                                   |
+-----------+-------------------+-------------------+-----------------+----------------------+
| Reduction | Lateral step (mm) | Req RPM for 1 m/s | Max speed (m/s) | Total rotation (deg) |
+-----------+-------------------+-------------------+-----------------+----------------------+
|         1 |              7.33 |               5.0 |            11.8 |                   30 |
|         2 |              3.66 |              10.0 |             5.9 |                   60 |
|         4 |              1.83 |              20.0 |            2.95 |                  120 |
|         8 |              0.92 |     